In [1]:
import nltk

In [2]:
from nltk.corpus import reuters   
reuters.fileids()[0:10]

[u'test/14826',
 u'test/14828',
 u'test/14829',
 u'test/14832',
 u'test/14833',
 u'test/14839',
 u'test/14840',
 u'test/14841',
 u'test/14842',
 u'test/14843']

# Tokenization and Normalization
Inbuilt tokenizer 'word_tokenize' was used.

All same tokens eleminated.

All tokens converted to lowercase characters.

Porter's stemmer and Snowball stemmer were explored.(Commented out because of poor performance.)






In [3]:
#tokenizer
from nltk.tokenize import word_tokenize
tokens=word_tokenize(reuters.raw())


In [4]:
len(tokens)

1548307

In [5]:
uniqueTokens=sorted(list(set(tokens))) #removing repeated tokens
len(uniqueTokens)


63353

In [6]:
del tokens

In [7]:
for i in xrange(len(uniqueTokens)):
    uniqueTokens[i]=uniqueTokens[i].lower()
uniqueTokens.sort()

In [8]:
#from nltk.stem.porter import *
#from nltk.stem.snowball import SnowballStemmer
#stemmer = PorterStemmer()
#stemmer=SnowballStemmer("english")
#for i in xrange(len(uniqueTokens)):
    #uniqueTokens[i]=stemmer.stem(uniqueTokens[i])
    
uniqueTokens=list(set(uniqueTokens))
len(uniqueTokens)                   #length after normalization

52711

In [9]:
uniqueTokens[0:10]

[u'mid-week',
 u'1,800',
 u'fawc',
 u'degussa',
 u'woods',
 u'hanging',
 u'blgr',
 u'localized',
 u'161.9',
 u'161.8']

# Indexing
Data structure used is dictionary (hash table) with keys as terms and values as posting lists implemented as variable length arrays.

For handelling wild card queries trie data structure is implemented (refer to the C++ code)

In [11]:
#initializing the inverted index with empty posting list for every term in thr corpus.
invIndex={}                
for tok in uniqueTokens:
    invIndex[tok]=[]        


In [106]:
##inverted Index creation. 
#data structure used: dictionary(hash table) with posting list as a variable sized array

for f in reuters.fileids():          #loop through each file
    txt = reuters.raw(f)              
    toks=[]                          #new list of tokens for each f
    toks=word_tokenize(txt)          #tokenize and normalize file in a way identical  
    for j in xrange(len(toks)):      #...as earlier to genenrate same terms
        toks[j]=toks[j].lower() 
    #from nltk.stem.snowball import SnowballStemmer    
    #stemmer = PorterStemmer()
    #stemmer=SnowballStemmer("english")
    #for i in xrange(len(toks)):
        #toks[i]=stemmer.stem(toks[i])   
    toks=list(set(toks))             #remove identical terms within the same document
    for tok in toks:
        if tok not in uniqueTokens:
            invIndex[tok]=[]
        
    for i in xrange(len(toks)):       #loop through each term of this doc
        curPosList=invIndex[toks[i]]      
        curPosList.append(f)        
        invIndex[toks[i]]=curPosList  #locate that term in invIndex and append doc ID of f to ot
    

In [ ]:
#deleting entries in the inv index with empty posting list if any
for key in invIndex.keys():
    lis=[]
    lis=invIndex[key]
    if len(lis)==0:
        del invIndex[key]
len(invIndex.keys())

In [44]:
#persisting inverted index to disk
import pickle
pickle.dump( invIndex, open( "save.p", "wb" ) )

In [10]:
#reading inverted index back from the disk
import pickle
invIndex = pickle.load( open( "save.p", "rb" ) )

In [11]:
invIndex['main'][0:5]        #first five terms of the posting list of a random term

[u'test/14849', u'test/14861', u'test/14959', u'test/15048', u'test/15095']

# Phase 2: TFIDF calculation and cosine similarity

In [47]:
#a new index that stores the tf value of every term and doc.
#structure is similar to that of inverted index, instead of posting list of docIDs
#...the value of hashtable is a list of tuples (docID,tf(t,d))

tfIndex={}
for key in invIndex.keys():
    tfIndex[key]=[]
    curPosLis=invIndex[key]
    for docID in curPosLis:
        tf=0
        txt = reuters.raw(docID)              
        toks=[]                    
        toks=word_tokenize(txt)            
        for j in xrange(len(toks)):      
            toks[j]=toks[j].lower()
        tf=toks.count(key)
        tfIndex[key].append([docID,tf])
        

In [13]:
#testing how posting list of a random word looks like in this index
tfIndex['main'][15:20]

[[u'test/16926', 2],
 [u'test/17038', 1],
 [u'test/17480', 1],
 [u'test/17486', 1],
 [u'test/17509', 1]]

In [45]:
#persisting tfindex index to disk
import pickle
pickle.dump( tfIndex, open( "tfindex.p", "wb" ) )


In [12]:
#reading tf index back from the disk
import pickle
tfIndex = pickle.load( open( "tfindex.p", "rb" ) )

In [16]:
N=len(reuters.fileids())
print N

10788


In [17]:
V=len(invIndex.keys())
print V

52708


In [18]:
#creating a dictionary of idfs:
#    ---keys: term
#    ---values: IDF==>log(N/(length of posting list of the corresponding terms))

import math
idf={}
for i in xrange(V):
    lis=invIndex[invIndex.keys()[i]]
    df=len(lis)
    if df!=0:
        idf[invIndex.keys()[i]]=math.log(N/df)
    else:
        idf[invIndex.keys()[i]]=0
    

In [19]:
idf['main']

3.7376696182833684

In [24]:
#writing the dictionary into a csv file 
import csv
with open('dict.csv', 'wb') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in idf.items():
       writer.writerow([key, value])

UnicodeEncodeError: 'ascii' codec can't encode character u'\xfc' in position 7: ordinal not in range(128)

In [23]:
len(idf.keys())

52708

In [20]:
import numpy as np
import math
U=np.zeros((V,N))
docID=[]
docID=reuters.fileids()
terms=[]
terms=invIndex.keys()

#creating tfidf matrix

for i in xrange(V):                         #for i th word in the inverted index
    posLis=tfIndex[invIndex.keys()[i]]      #access posting list of tfIndex
    for doc in posLis:                      #doc is a tuple of format (docID,tf)
        fileid=doc[0]
        tf=1+math.log(doc[1])
        r=i
        c=reuters.fileids().index(fileid)
        U[r,c]=tf*idf[terms[i]]

In [21]:
#reuters.fileids().index('test/14849')            
#11
#invIndex.keys().index('main')
#46810
U[46810,11]             #idf*tf=3.73*1 as expected

3.7376696182833684

In [22]:
reuters.fileids().index('test/16926') #'test/16926', 2]
#1190
U[46810,1190]                       #idf*tf=3.73*2 as expected

6.3284247760610528

In [23]:
#converts a vector represented by a list of numpy array into a unit vector
def unitNorm(lis):
    norm=0.0
    for i in xrange(len(lis)):
        norm+=lis[i]**2.0
    norm=norm**(0.5)
    for i in xrange(len(lis)):
        lis[i]=lis[i]/norm
        
    return lis

#computes cosine similarity between two unit normalised vectors
def cosineSim(vec1,vec2):
    #vec1=unitNorm(vec1)
    #vec2=unitNorm(vec2)
    if len(vec1)!=len(vec2):
        print 'vector length not same'
        return
    else:
        sim=0
        for i in xrange(len(vec2)):
            sim+=vec1[i]*vec2[i]
        return sim   

In [24]:
#unit normalize vector of each document
for i in xrange(N):
    vec=U[:,i]
    U[:,i]=unitNorm(vec)
    


In [25]:
U[46810,11]

0.062063432186458295

In [26]:
#saving the U matrix to disk
np.save('/tmp/123', U)

In [ ]:
#reading the matrix back from the disk
import numpy as np
np.load('/tmp/123.npy')

In [184]:
cosineSim([1,1,1],[1,1,1])

1.0000000000000002

In [96]:
V=len(invIndex.keys())

In [99]:
np.count_nonzero(qVector)
qVector.shape

(52708,)

In [28]:
U.shape

(52708, 10788)

In [29]:
U[:,0].shape

(52708,)

In [88]:
U.shape

(52708, 10788)

In [90]:
len(reuters.fileids())

10788

# Online Computation

Inverted index, tfidf matrix will be computed offline and saved before hand. When the user gives a query only the following code will run:


In [38]:
#computes cosine similarity between the query vector and document vectors and returns the docID
#of the top three similar docs in decreasing order of similarity
#(efficient implementation)
def cosineScore(q,queryVector):
    scores=[0 for x in xrange(N)]  #scores is a list of length N and accumulates the score of each
                                   #document (initialize to zero)
    
    for term in q:                  
        if term in invIndex.keys(): #walk through the posting list of each term in the query that is present in the corpus 
            posLis=invIndex[term]
            for doc in posLis:
                docIndex=reuters.fileids().index(doc)
                termIndex=invIndex.keys().index(term)
                scores[docIndex]+=U[termIndex,docIndex]*queryVector[termIndex]
                
    SCORES=scores
    #sort the scores list and return the last three docs
    maxLis=sorted(SCORES)               
    maxSim=maxLis[-1]
    Maxindex=scores.index(maxSim)
    doc=reuters.fileids()[Maxindex]     #most similar doc
    
    maxSim2=maxLis[-2]
    Maxindex=scores.index(maxSim2)
    doc1=reuters.fileids()[Maxindex]    #second most similar doc
    
    maxSim3=maxLis[-3]
    Maxindex=scores.index(maxSim3)
    doc2=reuters.fileids()[Maxindex]    #third most similar doc
    return [doc, doc1, doc2]
            

In [40]:
#preprocessing of query and conversion into vector

query=raw_input()  #take input query from the user

toks=word_tokenize(query)
for i in xrange(len(toks)):
    toks[i]=toks[i].lower()         #tokenize and normalize the query in the same way as the corpus
#stemmer = PorterStemmer()
#stemmer=SnowballStemmer("english")
#for i in xrange(len(toks)):
    #toks[i]=stemmer.stem(toks[i])
    

import numpy as np
qVector=np.zeros(V)             #store the query vector in a numpy array
for term in toks:
    if term in invIndex.keys():
        qVector[invIndex.keys().index(term)]=idf[term]
qVector=unitNorm(qVector)


cosineScore(toks,qVector)
#for example take the query "Pillsbury Company". The documents returned are [u'training/7106', u'training/1065', u'training/10073']

Pillsbury company


[u'training/7106', u'training/1065', u'training/10073']

In [32]:
print reuters.raw(cosineScore(toks,qVector)[0]) #even though it does not have the word company
#this is the first result because the word Pillsbury is a relatively rare word and has a higher
#idf

PILLSBURY CO 3RD QTR SHR 56 CTS VS 63 CTS

  PILLSBURY CO 3RD QTR SHR 56 CTS VS 63 CTS
  




In [33]:
print reuters.raw(cosineScore(toks,qVector)[1]) #similar to the first result, lower because of
#tf=1

PILLSBURY CO &lt;PSY> VOTES QUARTERLY DIVIDEND
  Qtly div 25 cts vs 25 cts prior qtr
      Pay 31 May
      Record 1 May
  




In [34]:
print reuters.raw(cosineScore(toks,qVector)[2]) #even though this has tf of pillsbury greater
#than the first two results, it has lesser weight because the effect is diluted due to length 
#normalization

PILLSBURY &lt;PSY> FILES FOR SECOND BURGER KING MLP
  The Pillsbury Co said it
  filed a registration statement with the Securities and Exchange
  Commission for the sale of limited partnership interests in a
  second master limited partnership of its Burger King unit's
  restaurant properties.
      Pillsbury said it expects the offering to yield 73-82 mln
  dlrs, resulting in an after-tax gain of 20-23 mln dlrs.
      A spokesman for Pillsbury said the company is aiming to get
  this after-tax gain in the fourth fiscal quarter ending in May.
      Pillsbury said the sale will occur as soon as practicable,
  considering the business and legal contigencies.
      The company said Burger King and another Pillsbury unit,
  QSV Properties, will be the master limited partnership's
  general partner.
      Pillsbury said it expects the interests to be sold to
  public investors and listed for trading on the New York Stock
  Exchange.
      Merrill Lynch will lead the underwriting, Pillsbury

# ADDITIONAL:

The next additional component leverages the word2vec model to retrieve documents intuitively similar to the querry.
About word2vec (from wikipedia):
Word2vec is a group of related models that are used to produce word embeddings. These models are shallow, two-layer neural networks that are trained to reconstruct linguistic contexts of words. Word2vec takes as its input a large corpus of text and produces a high-dimensional space (typically of several hundred dimensions), with each unique word in the corpus being assigned a corresponding vector in the space. Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space.


Paper on word2vec: https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

In [27]:
import gensim #optimized library that uses deep learning to generate skip gram model
              # to generate word2vec embeddings

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [29]:
import logging
#training the reuters corpus and forming a gensim model
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
sentences = reuters.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)

The model has now learnt word vectors for all the words in the corpus. The vectors learnt capture the context of the word and the words which appear in similar context have similar vectors.

In [30]:
model.most_similar('good',topn=5)

[(u'very', 0.9034268856048584),
 (u'positive', 0.8626291751861572),
 (u'bad', 0.8152326345443726),
 (u'significant', 0.8105217218399048),
 (u'better', 0.8077592849731445)]

In [31]:
model.most_similar('company',topn=5)

[(u'group', 0.7351366877555847),
 (u'transaction', 0.7068047523498535),
 (u'firm', 0.6975889205932617),
 (u'partnership', 0.6868374347686768),
 (u'acquisition', 0.6747386455535889)]

In [32]:
model.most_similar('industry')[0:5]

[(u'energy', 0.7945944666862488),
 (u'exporters', 0.7928650379180908),
 (u'field', 0.774497926235199),
 (u'shipping', 0.766197919845581),
 (u'sector', 0.7620868682861328)]

In [33]:
model.most_similar('main')[0:5]

[(u'key', 0.8374419212341309),
 (u'biggest', 0.8255422115325928),
 (u'affecting', 0.7950398921966553),
 (u'weakness', 0.7907974720001221),
 (u'world', 0.7907313108444214)]

In [34]:
model.most_similar('president')[0:5]

[(u'chairman', 0.957267165184021),
 (u'director', 0.9501000642776489),
 (u'chief', 0.9411285519599915),
 (u'executive', 0.913731575012207),
 (u'vice', 0.9005922079086304)]

When a user now gives in a query we will also append the the top 2 similar words to the query. This will increase our recall. But since reuters is a relatively small corpus, the similar words might not be relevant all the time. So in order to not harm the precision too much, the weights of these similar words are halved so that the original words of the query are given more importance.

In [35]:
def newCosineScore(q,queryVector):
    scores=[0 for x in xrange(N)]  #scores is a list of length N and accumulates the score of each
                                   #document (initialize to zero)
    
    for term in q:                  
        if term in invIndex.keys(): #walk through the posting list of each term in the query that is present in the corpus 
            posLis=invIndex[term]
            for doc in posLis:
                docIndex=reuters.fileids().index(doc)
                termIndex=invIndex.keys().index(term)
                scores[docIndex]+=U[termIndex,docIndex]*queryVector[termIndex]
            similarTerms=model.most_similar(term)[0:3]
            for simTerm in similarTerms: #also add weights to the words similar to the query vectors
                posLis=invIndex[simTerm[0]]
                for doc in posLis:
                    docIndex=reuters.fileids().index(doc)
                    termIndex=invIndex.keys().index(simTerm[0])
                    scores[docIndex]+=(U[termIndex,docIndex]*queryVector[termIndex])
                                          #Note that we have added the weight to be half of tfidf weight
                
    SCORES=scores
    #sort the scores list and return the last three docs
    maxLis=sorted(SCORES)               
    maxSim=maxLis[-1]
    Maxindex=scores.index(maxSim)
    doc=reuters.fileids()[Maxindex]     #most similar doc
    
    maxSim2=maxLis[-2]
    Maxindex=scores.index(maxSim2)
    doc1=reuters.fileids()[Maxindex]    #second most similar doc
    
    maxSim3=maxLis[-3]
    Maxindex=scores.index(maxSim3)
    doc2=reuters.fileids()[Maxindex]    #third most similar doc
    return [doc, doc1, doc2]

In [50]:
#preprocessing of query and conversion into vector

query=raw_input()

toks=word_tokenize(query)
for i in xrange(len(toks)):
    toks[i]=toks[i].lower()         #tokenize and normalize the query in the same way as the corpus
    

#stemmer = PorterStemmer()
#stemmer=SnowballStemmer("english")
#for i in xrange(len(toks)):
    #toks[i]=stemmer.stem(toks[i])
    
qVector=np.zeros(V)             #Old query vector containing non zero dimensions 
                                #...only for words in the query
for term in toks:
    if term in invIndex.keys():
        qVector[invIndex.keys().index(term)]=idf[term]
qVector=unitNorm(qVector)
    
    
#the new query vector qVector1 with appended dimensions of similar terms
qVector1=np.zeros(V)             
for term in toks:
    if term in invIndex.keys():
        qVector1[invIndex.keys().index(term)]=idf[term]
        
for term in toks:
    if term in invIndex.keys():
        lis=model.most_similar(term)[0:3]
        for item in lis:
            qVector1[invIndex.keys().index(item[0])]=idf[item[0]]/2 #halving weights of added terms                    
qVector1=unitNorm(qVector1)


lis=newCosineScore(toks,qVector1)

#consider the query example 'bad crops'

bad crops


In [51]:
lis #the new result

[u'training/1623', u'training/1299', u'test/21168']

In [52]:
cosineScore(toks,qVector) #result from the earlier model without word2vec

[u'test/21168', u'training/6494', u'test/15574']

In [53]:
reuters.raw('training/1623') #an intuitive article about crops going bad

u'IZVESTIA SAYS SOVIET WINTER CROPS NEED RESEEDING\n  The government daily Izvestia said a\n  considerable amount of Soviet winter crops need to be reseeded\n  and the state 1987 grain harvest target of 232 mln tonnes will\n  not be easy to fulfil.\n      Without giving figures, the newspaper said: "A considerable\n  part of the winter crops must be reseeded, but that creates\n  extra effort in the fields in spring."\n      The Soviet Union has previously said nine mln hectares of\n  winter grain will have to be reseeded because of winterkill.\n      A U.S. Department of Agriculture analyst in Washington has\n  said the figure of nine mln hectares would equal about 25 pct\n  of the total winter crop and would be the second highest\n  winterkill in 10 years.\n      "The planned task of bringing in no less than 232 mln tonnes\n  of grain is not simple," Izvestia said.\n      This week\'s sudden fall in temperatures has affected large\n  parts of the country and has caused fieldwork to st

In [54]:
reuters.raw('test/21168') #just a small article that has crops many times and completely
                          #ignores bad because of its small idf

u'U.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN RATE  CHANGES STARTING WITH 1988 CROPS\n\n  U.S. SENATE PANEL VOTES TO LIMIT COUNTY LOAN RATE  CHANGES STARTING WITH 1988 CROPS\n  \n\n'

# Archives
Earlier implementations that we considered but did not use because of inefficiency:

In [ ]:
#storing tfidf values in a dictionary of dictionary:
#termDoc is a dictionary which has all terms as keys and each key has a dictionary as its values
#each new dictionary has the doc ids in which the term appears and value is tf score of 
#that term and document.

termDoc={}
lis=[]
for i in xrange(len(invIndex.keys())):
    dic={}
    lis.append(dic)
    
for term in invIndex.keys():
    dic={}
    dic=lis[invIndex.keys().index(term)]                    #termDoc[term]=dic
    for doc in invIndex[term]:
        IDF=idf[term]
        TF=getTF(term,doc)
        if TF!=0:
            dic[doc]=IDF*(1+math.log(TF))
            
    termDoc[term]=dic

In [ ]:
#alternative general purpose methods for calculating tf and df for a single term and document
#(not used in the current system design)

def getDF(term):
    posLis=[]
    posLis=invIndex[term]
    return len(posLis)

def getTF(term,doc):
    txt = reuters.raw(f)
    toks=[]
    toks=word_tokenize(txt)
    for i in xrange(len(toks)):
        toks[i]=toks[i].lower()
    #stemmer = PorterStemmer()
    #stemmer=SnowballStemmer("english")
    #for i in xrange(len(toks)):
        #toks[i]=stemmer.stem(toks[i])
    return toks.count(term)


In [ ]:
#Naive implementation of storing TFIDF values in a N*V matrix


termDoc=[]
for i in xrange(V):
    lis=[]
    for j in xrange(N):
        tf=getTF(invIndex.keys()[i],reuters.fileids()[j])
        if tf!=0:
            lis.append(1+math.log(tf))
    termDoc.append(lis)

In [ ]:
#naive cosine similarity checking with each document vector
doc=0
maxSim=0

for i in xrange(N):
    docVec=U[:,i]
    simTemp=cosineSim(queryVector,docVec)
    if simTemp>maxSim:
        maxSim=simTemp
        doc=i

print reuters.fileids()[doc]  

In [ ]:
doc=0
maxSim=0

for term in queryVector:
    for doc in invIndex[term]:
        fileIndex=reuters.fileids().index(doc)
        docVec=U[:,fileIndex]
        simTemp=cosineSim(queryVector,docVec)
        if simTemp>maxSim:
            maxSim=simTemp
            DOC=doc

print DOC